In [33]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Literal
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

In [34]:
# create a state

class QuadState(TypedDict):
    a : int
    b: int
    c : int

    equation : str
    discriminant : float
    result : str
    


In [35]:
def show_equation(state : QuadState):
    equation = f'{state["a"]}x2{state["b"]}{state["c"]}'

    return {'equation' : equation}


In [36]:
def calculate_discriminant(state : QuadState):
    discriminant = state['b']**2 - 4 * state['a'] * state['c']

    return {'discriminant' : discriminant}

In [42]:
def real_roots(state:QuadState):
    root1 = (-state['b'] + state['discriminant']**0.5)/(2* state['a'])
    root2 = (-state['b'] - state['discriminant']**0.5)/(2* state['a'])

    result = f"roots are {root1} and {root2}"

    return {'result': result}

def repeated_roots(state: QuadState):
    root = - state['b']/(2 * state['a'])
    result = f" only repeating root is {root}"

    return {'result' : result}

def no_real_roots(state :QuadState):
    result = f'No real roots'

    return {'result' : result}

# conditional workflow
def check_condition(state:QuadState) -> Literal["real_roots","repeated_roots", "no_real_roots"]:
    if state['discriminant'] >0:
        return "real_roots"
    elif state['discriminant'] ==0:
        return "repeated_roots"
    else:
        return "no_real_roots"

In [43]:
graph = StateGraph(QuadState)

# create nodes
graph.add_node('show_equation', show_equation)
graph.add_node('calculate_discriminant', calculate_discriminant)
graph.add_node('real_roots',real_roots)
graph.add_node('repeated_roots',repeated_roots)
graph.add_node('no_real_roots',no_real_roots)

# create edges
graph.add_edge(START,'show_equation')
graph.add_edge('show_equation','calculate_discriminant')

graph.add_conditional_edges('calculate_discriminant',check_condition)
graph.add_edge('real_roots',END)
graph.add_edge('repeated_roots',END)
graph.add_edge('no_real_roots',END)




# compile
workflow = graph.compile()

In [45]:
intial_state = {'a' : 2 , 'b' :4, 'c' : 2}
final_state = workflow.invoke(intial_state)
print(final_state)

{'a': 2, 'b': 4, 'c': 2, 'equation': '2x242', 'discriminant': 0, 'result': ' only repeating root is -1.0'}
